# Reads assignment & Quality assessment

2021-04-15

In [2]:
# Import packages 
import sys
import numpy as np
import pandas as pd
import seaborn as sns
from natsort import natsorted
from scipy.io import loadmat, savemat
from skimage.filters import threshold_otsu
from skimage.color import label2rgb

# Customized packages 
from starmap.sequencing import *

In [4]:
# Get functions 

from functools import wraps
from time import time

# Timer
def timer(func):
    @wraps(func)
    def _time_it(*args, **kwargs):
        start = int(round(time() * 1000))
        try:
            return func(*args, **kwargs)
        finally:
            end_ = int(round(time() * 1000)) - start
            end_ = round(end_ / 1000, 4)
            print(f"Total execution time: {end_ if end_ > 0 else 0} s")
    return _time_it


# Trim reads 
@timer
def trim_reads(sample_dir, save_as=True):
    
    print(f"Trimming reads...")
    current_coordinates = np.loadtxt(os.path.join(sample_dir, 'trim.txt'), dtype=int, delimiter=',')
    current_dots = loadmat(os.path.join(sample_dir, 'merged_goodPoints_max3d.mat'))
    
    # Load reads from matlab data file
    bases = [str(i[0][0]) for i in current_dots["merged_reads"]]
    bases = np.array(bases)
    
    # Get reads location
    temp = current_dots["merged_points"]
    
    # Trim reads 
    to_remove = (temp[:, 0] < current_coordinates[0, 0]) | (temp[:, 0] > current_coordinates[1, 0]) | (temp[:, 1] < current_coordinates[0, 1]) | (temp[:, 1] > current_coordinates[1, 1])
    temp = temp[~to_remove, :]
    temp[:, 0] = temp[:, 0] - current_coordinates[0, 0]
    temp[:, 1] = temp[:, 1] - current_coordinates[0, 1]
    bases = bases[~to_remove]

    # Save trimmed reads
    if save_as:
        output_dict = {'trimmed_reads': bases, 'trimmed_points': temp}
        savemat(os.path.join(sample_dir, 'trimmed_goodPoints_max3d.mat'), output_dict)

    # Convert to 0 indexed and switch axis for python
    temp = temp[:, :2]
    points = np.zeros(temp.shape)
    points[:, 0] = np.round(temp[:, 1]-1)
    points[:, 1] = np.round(temp[:, 0]-1)
    print(f"Number of reads: {len(bases)}")
    
    return points, bases


# Load code book (genes.csv)
def load_genes(base_path):
    genes2seqs = {}
    seqs2genes = {}
    with open(os.path.join(base_path, "genes.csv"), encoding='utf-8-sig') as f:
        for l in f:
            fields = l.rstrip().split(",")
            curr_seg = "".join([str(s+1) for s in encode_SOLID(fields[1][::-1])])
            curr_seg = curr_seg[5:] + curr_seg[:4]
            # print(curr_seg)
            genes2seqs[fields[0]] = curr_seg
            seqs2genes[genes2seqs[fields[0]]] = fields[0]
            
    return genes2seqs, seqs2genes

## Setup

In [3]:
# Parameter
points_id = 'max3d'

# IO path 
base_path = './'
out_path = os.path.join(base_path, 'output')
if not os.path.exists(out_path):
    os.mkdir(out_path)
    
sample_dirs = [d for d in os.listdir(base_path) if d.startswith("AD")]
sample_dirs

['AD_mouse9494', 'AD_mouse9723', 'AD_mouse9735', 'AD_mouse9498']

## Run pipeline for individual sample

### Input

In [ ]:
# Iterate through each sample dir
current_dir = sample_dirs[0]
# print(current_dir)


print(f"Current sample: {current_dir}")


# Load reads 
points, bases = trim_reads(current_dir)

# Load genes
genes2seqs, seqs2genes = load_genes(base_path)

# Load dapi label (pi_seg)
dapi = load_label_image(os.path.join(current_dir, 'trimmed_images'), fname='pi_sum_seg_011.tif')
dapi.shape

# Get cell locations 
centroids = []

for i, region in enumerate(regionprops(dapi)):
    centroids.append(region.centroid)

centroids = np.array(centroids)

# Load 2D overlay image 
overlay = load_nissl_image(os.path.join(current_dir, 'trimmed_images'), fname="dots_pi_max_overlay.tif")

### Segmentation

In [ ]:
%%time
# Segmentation
seg_out_path = os.path.join(current_dir, 'segmentation')
if not os.path.exists(seg_out_path):
    os.mkdir(seg_out_path)

print("Gaussian & Thresholding")
blurred_overlay_seg = gaussian(overlay.astype(np.float), 10)
threhold = threshold_otsu(blurred_overlay_seg)

# otsu threshold 
blurred_overlay_seg = blurred_overlay_seg > threhold

# manual treshold 
# blurred_overlay_seg = gaussian(overlay.astype(np.float), 10) > 50

# dialation  
blurred_overlay_seg = binary_dilation(blurred_overlay_seg, selem=disk(10))

print("Assigning markers")
centroids = centroids.astype(int)
markers = np.zeros(blurred_nissl_seg.shape, dtype=np.uint8)
for i in range(centroids.shape[0]):
    x, y = centroids[i, :]
    if x < blurred_overlay_seg.shape[0] and y < blurred_overlay_seg.shape[1]:
        markers[x-1, y-1] = 1
markers = ndi.label(markers)[0]

print("Watershed")
labels = watershed(blurred_overlay_seg, markers, mask=blurred_overlay_seg)
labels_line = watershed(blurred_overlay_seg, markers, mask=blurred_overlay_seg, watershed_line=True)

print(f"Labeled {len(np.unique(labels)) - 1} cells")
plt.figure(figsize=(10,20))
plt.imshow(label2rgb(labels_line))

print(f"Saving files to {seg_out_path}")
tifffile.imsave(os.path.join(seg_out_path, "labeled_cells_line.tif"), labels_line.astype(np.uint16))
tifffile.imsave(os.path.join(seg_out_path, "labeled_cells.tif"), labels.astype(np.uint16))

In [ ]:
# Set figure size 
figsize=(labels_line.shape[1] / 1000 * 5, labels_line.shape[0] / 1000 * 5)

# Plot cell number 
t_size = 10
plt.figure(figsize=figsize)
plt.imshow(nissl)
for i, region in enumerate(regionprops(labels_line)):
    plt.plot(region.centroid[1], region.centroid[0], '.', color='red', markersize=4)
    plt.text(region.centroid[1], region.centroid[0], str(i), fontsize=t_size, color='red')

plt.axis('off')
plt.savefig(os.path.join(seg_out_path, "cell_nums.png"))
plt.clf()
plt.close()

In [ ]:
# Plot dots on segmentation mask
plt.figure(figsize=figsize)
plt.imshow(labels_line > 0, cmap='gray')
plt.plot(points[:, 1], points[:, 0], '.', color='red', markersize=1)
plt.axis('off')
points_seg_path = os.path.join(seg_out_path, "points_seg.png")
print(f"Saving points_seg.png")
plt.savefig(points_seg_path)
plt.clf()
plt.close()

In [ ]:
# Plot dots on overlay
plt.figure(figsize=figsize)
plt.imshow(nissl, cmap='gray')
plt.plot(points[:, 1], points[:, 0], '.', color='red', markersize=1)
plt.axis('off')
points_seg_path = os.path.join(seg_out_path, "points_nissl.png")
print(f"Saving points_nissl.png")
plt.savefig(points_seg_path)
plt.clf()
plt.close()

### Reads assignment

In [ ]:
%%time
# Reads assignment to cell
expr_out_path = os.path.join(out_path, current_dir)
if not os.path.exists(expr_out_path):
    os.mkdir(expr_out_path)
        
points = points.astype(int)
reads_assignment = labels[points[:, 0], points[:, 1]]
    
cell_locs = []
total_cells = len(np.unique(labels)) - 1
areas = []

gene_seqs = seqs2genes.keys()
cell_by_barcode = np.zeros((total_cells, len(gene_seqs)))
gene_seq_to_index = {}  # map from sequence to index into matrix

for i, k in enumerate(gene_seqs):
    gene_seq_to_index[k] = i
    
# Iterate through cells
print('Iterate cells...')
for i, region in enumerate(regionprops(labels)):
    # print(region.label)
    areas.append(region.area)
    cell_locs.append(region.centroid)
    
    assigned_reads = bases[np.argwhere(reads_assignment == region.label).flatten()]
    for j in assigned_reads:
        if j in gene_seq_to_index:
            cell_by_barcode[i, gene_seq_to_index[j]] += 1
    
     
# Construct output
cell_locs = np.array(cell_locs).astype(int)
curr_meta = pd.DataFrame({'sample': current_dir, 'area': areas,
                          'x':cell_locs[:, 1], 'y':cell_locs[:, 0]})

with open(os.path.join(expr_out_path, "log.txt"), 'w') as f:
    msg = "{:.2%} percent [{} out of {}] reads were assigned to {} cells".format(cell_by_barcode.sum()/len(bases), cell_by_barcode.sum(), len(bases), total_cells)
    print(msg)
    f.write(msg)
np.savetxt(os.path.join(expr_out_path, "cell_barcode_count.csv"), cell_by_barcode.astype(np.int), delimiter=',', fmt="%d")
cell_barcode_names = pd.DataFrame({'seq': list(seqs2genes.keys()), 'gene': list(seqs2genes.values())})
cell_barcode_names.to_csv(os.path.join(expr_out_path, "cell_barcode_names.csv"), header=False)
curr_meta.to_csv(os.path.join(expr_out_path, "meta.csv"))

In [ ]:
# Preview current metadata
curr_meta.head()

In [ ]:
# Plot area distribution
sns.distplot(areas)

### Reads pattern visualization

In [ ]:
# Get assigned reads 
assigned_index = np.argwhere(reads_assignment != 0).flatten()
assigned_bases = bases[assigned_index]
assigned_points = points[assigned_index, :]

In [ ]:
# Get reads of specific gene
gene = 'PPP1R9B'
curr_index = np.argwhere(assigned_bases == genes2seqs[gene]).flatten()
curr_points = assigned_points[curr_index, :]
print(f"Number of reads: {curr_points.shape[0]}")

In [ ]:
# Plot dots on segmentation mask
plt.figure(figsize=figsize)
plt.imshow(nissl, cmap='gray')
plt.plot(curr_points[:, 1], curr_points[:, 0], '.', color='red', markersize=1)
plt.axis('off')
plt.show()

In [ ]:
# Get read quantification for each gene after read assignemnt and got the top-20
per_gene_expr = pd.DataFrame({'gene': list(seqs2genes.values()), 'expr': cell_by_barcode.sum(axis=0)})
per_gene_expr = per_gene_expr.sort_values('expr', ascending=False, ignore_index=True)

# Get top 20 genes & Curated markers 
top20 = per_gene_expr.head(20).gene.to_list()
curated = ['SLC17A7', 'CUX2', 'RORB', 'SULF2', 'PCP4',
          'GAD1', 'PVALB', 'SST', 'NPY', 'VIP', 'MBP', 'MOBP']
selected_genes = top20 + curated
selected_genes

In [ ]:
# Generate reads pattern plot for all the selected genes 
expr_figure_out_path = os.path.join(expr_out_path, 'figures')
if not os.path.exists(expr_figure_out_path):
    os.mkdir(expr_figure_out_path)
    
for i, gene in enumerate(selected_genes):
    
    curr_index = np.argwhere(assigned_bases == genes2seqs[gene]).flatten()
    curr_points = assigned_points[curr_index, :]
    n_reads = curr_points.shape[0]

    # Plot
    plt.figure(figsize=figsize)
    plt.imshow(nissl, cmap='gray')
    plt.plot(curr_points[:, 1], curr_points[:, 0], '.', color='red', markersize=1)
    plt.axis('off')
    expr_figure_path = os.path.join(expr_figure_out_path, f"{i+1}.{gene}_{n_reads}.png")
    plt.savefig(expr_figure_path)
    plt.clf()
    plt.close()

## Generate complete matrix

*run this after finishing assignments for all samples*

In [ ]:
# Construct complete matrix
cell_by_gene_complete = None
meta_complete = None

for i, d in enumerate(sample_dirs):
    print(f"Loading sample: {d}")
    current_expr_path = os.path.join(out_path, d)
    current_expr = np.loadtxt(os.path.join(current_expr_path, "cell_barcode_count.csv"), dtype=int, delimiter=',')
    current_meta = pd.read_csv(os.path.join(current_expr_path, "meta.csv"))
    
    # add to complete matrix
    if cell_by_gene_complete is not None:
        cell_by_gene_complete = np.concatenate((cell_by_gene_complete, current_expr))
    else:
        cell_by_gene_complete = current_expr
        
    if meta_complete is not None:
        meta_complete = pd.concat([meta_complete, current_meta])
    else:
        meta_complete = current_meta
        
np.savetxt(os.path.join(out_path, "complete_cell_barcode_count.csv"), cell_by_gene_complete.astype(np.int), delimiter=',', fmt="%d")
meta_complete = meta_complete.reset_index(drop=True)
meta_complete = meta_complete.rename(columns={"Unnamed: 0": "orig_index"})
meta_complete.to_csv(os.path.join(out_path, "complete_meta.csv"))